<a href="https://colab.research.google.com/github/YaninaK/predictive-maintenance/blob/main/notebooks/02_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Модель раннего обнаружения неисправностей промышленного оборудования
## Исследователский анализ данных

#### [Чтение и предварительная очистка данных](https://colab.research.google.com/drive/1UHlDhDV11MuN3C4U-KqpJP736KmudnyA?usp=sharing)

ОПИСАНИЕ ЗАДАЧИ

Разработать модель, определяющую возможность возникновения нештатной работы оборудования на временном горизонте не менее 3 часов до аварийного простоя.

[РЕСУРСЫ](https://drive.google.com/file/d/1jrbfHULbZuCnwJQwNllQUFlCGpR_lHDc/view?usp=sharing)

Наборы обезличенных данных из внутренних систем ПАО «Северсталь», содержащие:
* Перечень нештатных событий, приведших к остановке линии;
* Перечень нештатных событий, повлекших деградацию функций агрегатов;
* Перечень сигналов с датчиков, контроллеров.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!git clone https://github.com/YaninaK/predictive-maintenance.git -q
!pip install -r predictive-maintenance/requirements_Colab.txt -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [3]:
%cd /content/predictive-maintenance

/content/predictive-maintenance


In [4]:
!git pull -q

In [5]:
import sys
import os

sys.path.append(os.getcwd())
sys.path.append(os.path.join(os.getcwd(), "src", "predictive_maintenance"))   

In [6]:
import pandas as pd
import numpy as np

from data.EDA_utilities import get_y_summary, load_X, load_y 

import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
PATH = '/content/drive/MyDrive/ML_projects/02_Predictive_maintenance/'
FOLDER_1 = "data/02_intermediate/"
FOLDER_2 = "data/03_primary/"

### 1. Загрузка данных

In [9]:
messages = pd.read_parquet(PATH + FOLDER_1 + "messages_unified.parquet")

print(f'messages.shape = {messages.shape}\n')
messages.head(2)

messages.shape = (981, 11)



,МАШИНА,ИМЯ_МАШИНЫ,ТЕХ_МЕСТО,НАЗВАНИЕ_ТЕХ_МЕСТА,ВИД_СООБЩЕНИЯ,ОПИСАНИЕ,ДАТА_НАЧАЛА_НЕИСПРАВНОСТИ,ДАТА_УСТРАНЕНИЯ_НЕИСПРАВНОСТИ,ТЕКСТ_ГРУППЫ_КОДОВ,equipment,unified_name
390,AA2/006-006,ЭКСГАУСТЕР А/М №9,AA2/006-006-002-008,ЗАПОРНАЯ АРМАТУРА ЭКСГАУСТЕРА №9,M3,неисправен двигатель,2019-01-21 00:00:00,2019-02-25,None,9,ЗАПОРНАЯ АРМАТУРА ЭКСГАУСТЕРА №
391,CH-AGP-AG2/011-005,ЭКСГАУСТЕР А/М №9,CH-AGP-AG2/011-005-002,МАСЛОСТАНЦИЯ ЖИДКОЙ СМАЗКИ ЭКСГ. №9,M3,неисправен двигатель,2019-01-21 12:26:08,2019-02-25,None,9,МАСЛОСТАНЦИЯ ЖИДКОЙ СМАЗКИ ЭКСГ_ №


In [10]:
unified_tech_places = pd.read_parquet(
    PATH + FOLDER_1 + "unified_tech_places.parquet"
)
print(f'unified_tech_places.shape = {unified_tech_places.shape}\n')
unified_tech_places.head(2)

unified_tech_places.shape = (175, 3)



,equipment,description,unified_name
0,9,9_ЗАПОРНАЯ АРМАТУРА ЭКСГАУСТЕРА №9,ЗАПОРНАЯ АРМАТУРА ЭКСГАУСТЕРА №
1,9,9_МАСЛОСТАНЦИЯ ЖИДКОЙ СМАЗКИ ЭКСГ_ №9,МАСЛОСТАНЦИЯ ЖИДКОЙ СМАЗКИ ЭКСГ_ №


In [11]:
test_intervals = pd.read_parquet(
    PATH + FOLDER_1 + "test_intervals.parquet"
)
print(f'test_intervals.shape = {test_intervals.shape}\n')
test_intervals.head(2)

test_intervals.shape = (189, 4)



,start,finish,machine,tm
0,2022-01-07 09:05:16,2022-01-07 14:05:15,NaN,NaN
1,2022-02-25 03:44:52,2022-02-25 08:15:03,NaN,NaN


In [12]:
i = 4
X_train = load_X(i, path=PATH)

print(f'X_train.shape = {X_train.shape}\n')
X_train.head(2)

X_train.shape = (1555839, 16)



,4 ВИБРАЦИЯ НА ОПОРЕ 1,4 ВИБРАЦИЯ НА ОПОРЕ 2,4 ВИБРАЦИЯ НА ОПОРЕ 3,4 ВИБРАЦИЯ НА ОПОРЕ 3 ПРОДОЛЬНАЯ,4 ВИБРАЦИЯ НА ОПОРЕ 4,4 ВИБРАЦИЯ НА ОПОРЕ 4 ПРОДОЛЬНАЯ,4 ДАВЛЕНИЕ МАСЛА В СИСТЕМЕ,4 ТЕМПЕРАТУРА МАСЛА В МАСЛОБЛОКЕ,4 ТЕМПЕРАТУРА МАСЛА В СИСТЕМЕ,4 ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 1,4 ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 2,4 ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 3,4 ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 4,4 ТОК РОТОРА 1,4 ТОК РОТОРА 2,4 ТОК СТАТОРА
dt,,,,,,,,,,,,,,,,
2019-01-16 13:21:00,1.562818,0.743243,0.656307,1.632879,3.274178,0.738962,119.375153,40.369577,33.369792,41.427914,39.751736,39.862907,44.595902,363.321395,363.321395,243.301550
2019-01-16 13:22:00,1.557138,0.737997,0.660079,1.621047,3.220093,0.725388,119.395694,40.332575,33.506294,41.509807,39.725041,39.953993,44.736912,362.849194,362.849194,242.917778


In [13]:
y = load_y(i, path=PATH)

print(f'y.shape = {y.shape}\n')
y.head(2)

y.shape = (1555839, 23)



,4_ВК 310С ВИБРОПРЕОБРАЗОВАТЕЛЬ ЭКСГ_№4 Т_1,4_ГСМ ЭКСГ_ №4,4_ЗАДВИЖКА ЭКСГ_ №4,4_ЗАП_ И РЕГ_ АРМАТУРА ЭКСГ_№4,4_КЛ1 ТР№4 ДО ЭД ЭКСГАУСТЕРА №4,4_КЛ2 ТР№4 ДО ЭД ЭКСГАУСТЕРА №4,4_МАСЛОНАСОС РАБОЧИЙ ЭКСГ_ №4,4_МАСЛООХЛАДИТЕЛЬ М-05-1 ЭКСГ_ №4,4_МАСЛОПРОВОДЫ ЭКСГ №4,4_ПОДШИПНИК ОПОРНО-УПОРНЫЙ ЭКСГ_ №4,...,4_РЕДУКТОР ГАЗ_ ЗАДВИЖКИ ЭКСГ_ №4,4_РОТОР ЭКСГ_ №4,4_ТИРИСТ_ ВОЗБУДИТЕЛЬ ВТ-РЭМ-400 ЭКСГ4 ВУ1,4_ТР-Р ТМ-4000-10/6 ЭКСГ_ №4,4_ТСМТ-101-010-50М-400 ТЕРМОПР_ПОДШ_Т_1,4_УЛИТА ЭКСГ_ №4,4_ЭКСГАУСТЕР А/М №4,4_ЭЛ/ДВИГАТЕЛЬ ГАЗ_ ЗАДВИЖКИ ЭКСГ_ №4,4_ЭЛЕКТРОАППАРАТУРА ЭКСГ_ №4,4_ЭЛЕКТРОДВИГАТЕЛЬ ДСПУ-140-84-4 ЭКСГ_ №4
dt,,,,,,,,,,,,,,,,,,,,,
2019-01-16 13:21:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-16 13:22:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
X_test = load_X(i, path=PATH, prefix="X_test")

print(f'X_test.shape = {X_test.shape}\n')
X_test.head(2)

X_test.shape = (668161, 16)



,4 ВИБРАЦИЯ НА ОПОРЕ 1,4 ВИБРАЦИЯ НА ОПОРЕ 2,4 ВИБРАЦИЯ НА ОПОРЕ 3,4 ВИБРАЦИЯ НА ОПОРЕ 3 ПРОДОЛЬНАЯ,4 ВИБРАЦИЯ НА ОПОРЕ 4,4 ВИБРАЦИЯ НА ОПОРЕ 4 ПРОДОЛЬНАЯ,4 ДАВЛЕНИЕ МАСЛА В СИСТЕМЕ,4 ТЕМПЕРАТУРА МАСЛА В МАСЛОБЛОКЕ,4 ТЕМПЕРАТУРА МАСЛА В СИСТЕМЕ,4 ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 1,4 ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 2,4 ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 3,4 ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 4,4 ТОК РОТОРА 1,4 ТОК РОТОРА 2,4 ТОК СТАТОРА
dt,,,,,,,,,,,,,,,,
2022-01-01 00:00:00,0.758333,1.180000,0.360,1.906000,2.075,1.416667,134.444,NaN,31.235,42.595,43.540,30.830,34.394,247.51,247.51,271.153333
2022-01-01 00:01:00,0.738000,1.211667,0.374,1.913333,1.780,1.291667,134.420,33.26,31.100,42.595,43.405,31.235,34.610,247.39,247.39,271.195000


## 2. EDA

### 2.1. Разметка

#### 2.1.1 Messages

In [15]:
t0 = pd.Timestamp("1970-01-01")
period = pd.Timedelta('1T')
messages['start_M'] = (
    (messages['ДАТА_НАЧАЛА_НЕИСПРАВНОСТИ']  - t0) // period * period + t0
)
messages.loc[536, 'start_M'] = pd.Timestamp('2019-04-28 10:05:00')
messages.head(2)

,МАШИНА,ИМЯ_МАШИНЫ,ТЕХ_МЕСТО,НАЗВАНИЕ_ТЕХ_МЕСТА,ВИД_СООБЩЕНИЯ,ОПИСАНИЕ,ДАТА_НАЧАЛА_НЕИСПРАВНОСТИ,ДАТА_УСТРАНЕНИЯ_НЕИСПРАВНОСТИ,ТЕКСТ_ГРУППЫ_КОДОВ,equipment,unified_name,start_M
390,AA2/006-006,ЭКСГАУСТЕР А/М №9,AA2/006-006-002-008,ЗАПОРНАЯ АРМАТУРА ЭКСГАУСТЕРА №9,M3,неисправен двигатель,2019-01-21 00:00:00,2019-02-25,None,9,ЗАПОРНАЯ АРМАТУРА ЭКСГАУСТЕРА №,2019-01-21 00:00:00
391,CH-AGP-AG2/011-005,ЭКСГАУСТЕР А/М №9,CH-AGP-AG2/011-005-002,МАСЛОСТАНЦИЯ ЖИДКОЙ СМАЗКИ ЭКСГ. №9,M3,неисправен двигатель,2019-01-21 12:26:08,2019-02-25,None,9,МАСЛОСТАНЦИЯ ЖИДКОЙ СМАЗКИ ЭКСГ_ №,2019-01-21 12:26:00


In [16]:
messages.shape

(981, 12)

In [17]:
messages['ВИД_СООБЩЕНИЯ'].value_counts()

M3    898
M1     83
Name: ВИД_СООБЩЕНИЯ, dtype: int64

* M1 - остановки эксгаустеров
* M3 - аномалии в работе эксгаустеров

##### M1

In [18]:
pd.pivot_table(
    messages[messages['ВИД_СООБЩЕНИЯ'] == 'M1'], 
    index='ТЕКСТ_ГРУППЫ_КОДОВ', 
    columns='equipment', 
    values='ОПИСАНИЕ', 
    aggfunc='count',
    margins=True,   
)

equipment,4,5,6,7,8,9,All
ТЕКСТ_ГРУППЫ_КОДОВ,,,,,,,
ВНЕШНИЕ ОРГАНИЗАЦИИ,NaN,1.0,NaN,NaN,1.0,NaN,2
ТЕХНИЧЕСКИЕ НЕПЛАНОВЫЕ,17.0,19.0,6.0,11.0,12.0,4.0,69
ТЕХНИЧЕСКИЕ ПЛАНОВЫЕ,2.0,3.0,1.0,2.0,NaN,NaN,8
ТЕХНОЛОГИЧЕСКИЕ НЕПЛАНОВЫЕ,1.0,NaN,NaN,1.0,1.0,NaN,3
ТЕХНОЛОГИЧЕСКИЕ ПЛАНОВЫЕ,NaN,NaN,1.0,NaN,NaN,NaN,1
All,20.0,23.0,8.0,14.0,14.0,4.0,83


Прогнозировать имеет смысл только на неплановые остановки оборудования: ТЕХНИЧЕСКИЕ НЕПЛАНОВЫЕ и ТЕХНОЛОГИЧЕСКИЕ НЕПЛАНОВЫЕ.

In [19]:
cond_1 = messages['ТЕКСТ_ГРУППЫ_КОДОВ'] == 'ТЕХНИЧЕСКИЕ НЕПЛАНОВЫЕ' 
cond_2 = messages['ТЕКСТ_ГРУППЫ_КОДОВ'] == 'ТЕХНОЛОГИЧЕСКИЕ НЕПЛАНОВЫЕ'

pd.pivot_table(
    messages[cond_1 | cond_2], 
    index='unified_name', 
    columns='equipment', 
    values='ДАТА_НАЧАЛА_НЕИСПРАВНОСТИ', 
    aggfunc='count',
    margins=True,    
)

equipment,4,5,6,7,8,9,All
unified_name,,,,,,,
ЗАДВИЖКА ЭКСГ_ №,4.0,1.0,6.0,NaN,NaN,NaN,11
КОРПУС ЭКСГ_ №,NaN,NaN,NaN,2.0,NaN,NaN,2
ПОДШИПНИК ОПОРНО-УПОРНЫЙ ЭКСГ_ №,1.0,NaN,NaN,NaN,NaN,NaN,1
ПОДШИПНИК ОПОРНЫЙ №1,NaN,5.0,NaN,NaN,NaN,NaN,5
ПОДШИПНИК ОПОРНЫЙ №2,3.0,NaN,NaN,1.0,NaN,NaN,4
РОТОР ЭКСГ_ №,5.0,4.0,NaN,4.0,7.0,NaN,20
ТИРИСТ_ ВОЗБУДИТЕЛЬ ВТ-РЭМ-400 ЭКСГ ВУ1,NaN,4.0,NaN,NaN,NaN,NaN,4
ТИРИСТ_ ВОЗБУДИТЕЛЬ ВТ-РЭМ-400 ЭКСГ ВУ2,NaN,1.0,NaN,NaN,NaN,NaN,1
ТИРИСТОРНЫЙ ВОЗБУДИТЕЛЬ ТВ-400 ЭКСГ ВУ1,NaN,NaN,NaN,NaN,2.0,NaN,2


* Наиболшее число остановок на четвертом и пятом эксгаустерах, наименьшее - на шестом и девятом.
* Причины остановок между эксгаустерами существенно отличаются, но больше половины остановок произошло из-за неполадок с ротором, задвижкой и электродвигателем.

##### M3

In [20]:
pd.pivot_table(
    messages[messages['ВИД_СООБЩЕНИЯ'] == 'M3'], 
    index='unified_name', 
    columns='equipment', 
    values='ДАТА_НАЧАЛА_НЕИСПРАВНОСТИ', 
    aggfunc='count',
    margins=True,    
)

equipment,4,5,6,7,8,9,All
unified_name,,,,,,,
ВК 310С ВИБРОПРЕОБРАЗОВАТЕЛЬ ЭКСГ_№ Т_1,1.0,2.0,NaN,NaN,NaN,1.0,4
ВК 310С ВИБРОПРЕОБРАЗОВАТЕЛЬ ЭКСГ_№ Т_2,NaN,3.0,NaN,1.0,2.0,NaN,6
ВК 310С ВИБРОПРЕОБРАЗОВАТЕЛЬ ЭКСГ_№ Т_3,NaN,4.0,NaN,4.0,2.0,NaN,10
ВК 310С ВИБРОПРЕОБРАЗОВАТЕЛЬ ЭКСГ_№ Т_4,NaN,NaN,NaN,1.0,2.0,NaN,3
ГСМ ЭКСГ_ №,5.0,5.0,NaN,5.0,6.0,3.0,24
ДВИГАТЕЛЬ ПУСКОВОГО МАСЛОНАСОСА ЭКСГ_ №,NaN,2.0,NaN,NaN,NaN,NaN,2
ЗАДВИЖКА ЭКСГ_ №,12.0,22.0,12.0,8.0,12.0,2.0,68
ЗАП_ И РЕГ_ АРМАТУРА ЭКСГ_№,1.0,NaN,NaN,NaN,NaN,NaN,1
ЗАПОРНАЯ АРМАТУРА ЭКСГАУСТЕРА №,NaN,3.0,NaN,NaN,2.0,1.0,6


* Больше половины всех аномалий зафиксировано в корпусах, роторах, редукторах, электродвигателях и задвижках эксгаустеров.
* Больше половины неполадок с корпусами приходилось на шестой и седьмой эксгаустеры.
* Больше половины неполадок с роторами произошли на седьмом и восьмом эксгаустерах.
* 40% аномалий в работе редукторов зафиксировано на четвертом эксгаустере.
* 32% аномалий в работе электродвигателей и 35% - в работе задвижек случились на пятом эксгаустере.

In [21]:
cond_1 = messages['ВИД_СООБЩЕНИЯ'] == 'M3'
cond_2 = messages['ДАТА_УСТРАНЕНИЯ_НЕИСПРАВНОСТИ'].isnull()

messages[cond_1 & cond_2].shape

(130, 12)

* У 130 аномалий из 898 нет данных об времени устранения неисправности.

#### 2.1.2. Анализ разметки остановок и аномалий на y_train

In [22]:
y_spec_1 = pd.pivot_table(
    unified_tech_places, 
    index='unified_name', 
    columns='equipment', 
    values='description', 
    aggfunc='count',
    margins=True,    
)
print(f'y_spec_1.shape = {y_spec_1.shape}\n')
y_spec_1

y_spec_1.shape = (52, 7)



equipment,4,5,6,7,8,9,All
unified_name,,,,,,,
ВК 310С ВИБРОПРЕОБРАЗОВАТЕЛЬ ЭКСГ_№ Т_1,1.0,1.0,NaN,NaN,NaN,1.0,3
ВК 310С ВИБРОПРЕОБРАЗОВАТЕЛЬ ЭКСГ_№ Т_2,NaN,1.0,1.0,1.0,1.0,NaN,4
ВК 310С ВИБРОПРЕОБРАЗОВАТЕЛЬ ЭКСГ_№ Т_3,NaN,1.0,1.0,1.0,1.0,NaN,4
ВК 310С ВИБРОПРЕОБРАЗОВАТЕЛЬ ЭКСГ_№ Т_4,NaN,NaN,NaN,1.0,1.0,NaN,2
ГАЗОВАЯ ЗАДВИЖКА ЭКСГАУСТЕРА А/М №,NaN,1.0,1.0,NaN,NaN,NaN,2
ГСМ ЭКСГ_ №,1.0,1.0,NaN,1.0,1.0,1.0,5
ДВИГАТЕЛЬ ПУСКОВОГО МАСЛОНАСОСА ЭКСГ_ №,NaN,1.0,1.0,NaN,NaN,1.0,3
ДВИГАТЕЛЬ РЕЗЕРВНОГО МАСЛОНАСОСА ЭКСГ_№,NaN,1.0,NaN,NaN,NaN,NaN,1
ЗАДВИЖКА ЭКСГ_ №,1.0,1.0,1.0,1.0,1.0,1.0,6


* Разметка на y_train доступна менее чем по 60% технических мест.

In [23]:
regenerate = False
if regenerate:
  y_summary = get_y_summary(unified_tech_places, path=PATH)
  y_summary.to_parquet(PATH + FOLDER_1 + "y_summary.parquet", compression="gzip") 
else:
  y_summary = pd.read_parquet(PATH + FOLDER_1 + "y_summary.parquet")

In [24]:
y_spec_2 = pd.pivot_table(
    y_summary, 
    index='unified_name', 
    columns='equipment', 
    values='start_M', 
    aggfunc='count',
    margins=True,    
)
print(f'y_spec_2.shape = {y_spec_2.shape}\n')
y_spec_2

y_spec_2.shape = (49, 7)



equipment,4,5,6,7,8,9,All
unified_name,,,,,,,
ВК 310С ВИБРОПРЕОБРАЗОВАТЕЛЬ ЭКСГ_№ Т_1,1.0,1.0,NaN,NaN,NaN,1.0,3
ВК 310С ВИБРОПРЕОБРАЗОВАТЕЛЬ ЭКСГ_№ Т_2,NaN,2.0,NaN,1.0,2.0,NaN,5
ВК 310С ВИБРОПРЕОБРАЗОВАТЕЛЬ ЭКСГ_№ Т_3,NaN,1.0,NaN,3.0,1.0,NaN,5
ВК 310С ВИБРОПРЕОБРАЗОВАТЕЛЬ ЭКСГ_№ Т_4,NaN,NaN,NaN,1.0,1.0,NaN,2
ГАЗОВАЯ ЗАДВИЖКА ЭКСГАУСТЕРА А/М №,NaN,1.0,1.0,NaN,NaN,NaN,2
ГСМ ЭКСГ_ №,5.0,1.0,NaN,5.0,5.0,3.0,19
ДВИГАТЕЛЬ ПУСКОВОГО МАСЛОНАСОСА ЭКСГ_ №,NaN,2.0,NaN,NaN,NaN,NaN,2
ЗАДВИЖКА ЭКСГ_ №,14.0,11.0,9.0,4.0,5.0,2.0,45
ЗАП_ И РЕГ_ АРМАТУРА ЭКСГ_№,1.0,NaN,NaN,NaN,NaN,NaN,1


In [25]:
no_label = set(y_spec_1.index) - set(y_spec_2.index)
no_label 

{'ДВИГАТЕЛЬ РЕЗЕРВНОГО МАСЛОНАСОСА ЭКСГ_№',
 'РЕГУЛИРУЮЩАЯ АППАРАТУРА ЭКСГАУСТЕРА №',
 'ЭКСГАУСТЕР Н-8000 А/М №'}

* Аномалии работы двигателя резервного маслонасоса, эксгаустера Н-8000 (эксг 5) и регулирующей аппаратуры эксгаустера (эксг 7) не имеют разметки.
* Значительная часть разметки из messages не попала в y_train: 898 vs 395. 
* Разметку для модели будем создавать на основе messages.

### 2.2. Обучающая выборка

In [26]:
i = 7
X = load_X(i, path=PATH)
X.describe(percentiles=[0.01, 0.25, 0.5, 0.75, 0.99]).T

,count,mean,std,min,1%,25%,50%,75%,99%,max
7 ВИБРАЦИЯ НА ОПОРЕ 1,1520107.0,2.551924,1.744509,0.0,0.042500,1.208333,2.140000,3.365389,7.845665,17.015000
7 ВИБРАЦИЯ НА ОПОРЕ 2,1509967.0,1.122693,0.637660,0.0,0.070000,0.730000,1.043639,1.382000,3.210000,15.946667
7 ВИБРАЦИЯ НА ОПОРЕ 3,1513131.0,1.773666,1.302607,0.0,0.055250,0.818333,1.378746,2.665690,5.493333,11.705000
7 ВИБРАЦИЯ НА ОПОРЕ 3 ПРОДОЛЬНАЯ,1520211.0,1.791713,1.495168,0.0,0.065139,1.021332,1.284449,2.322819,7.932975,24.428000
7 ВИБРАЦИЯ НА ОПОРЕ 4,1512204.0,2.284975,1.904846,0.0,0.074667,0.964000,1.589185,3.283083,9.156162,50.000000
7 ВИБРАЦИЯ НА ОПОРЕ 4 ПРОДОЛЬНАЯ,1513171.0,1.772263,1.344033,0.0,0.040889,0.843333,1.340889,2.274722,6.425000,15.851000
7 ДАВЛЕНИЕ МАСЛА В СИСТЕМЕ,1504385.0,133.078282,36.485401,0.0,1.100000,114.301745,130.008790,154.863333,219.890947,348.912000
7 ТЕМПЕРАТУРА МАСЛА В МАСЛОБЛОКЕ,1461431.0,41.172552,8.149304,0.0,16.876000,37.264869,41.780000,45.471503,55.598827,100.000000
7 ТЕМПЕРАТУРА МАСЛА В СИСТЕМЕ,1455034.0,31.203555,8.335098,0.0,8.272000,26.284000,31.995464,37.028319,48.270000,55.980000
7 ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 1,1462803.0,46.670447,7.708206,0.0,17.850000,43.000000,47.523183,51.700230,60.032500,67.745000


### 2.3. Тестовая выборка


##### 2.3.1. test_intervals

Для этих интервалов необходимо предсказать наличие неисправности. В случае предсказанной неисправности, определить тип и время начала развития и наступления неисправности.

In [27]:
test_intervals.head()

,start,finish,machine,tm
0,2022-01-07 09:05:16,2022-01-07 14:05:15,NaN,NaN
1,2022-02-25 03:44:52,2022-02-25 08:15:03,NaN,NaN
2,2022-03-11 06:06:36,2022-03-11 16:26:01,NaN,NaN
3,2022-03-15 04:27:52,2022-03-15 10:01:46,NaN,NaN
4,2022-03-20 15:07:30,2022-03-20 19:12:11,NaN,NaN


In [28]:
test_intervals.shape

(189, 4)

In [29]:
test_intervals['start'].min(), test_intervals['finish'].max()

(Timestamp('2022-01-01 10:54:53'), Timestamp('2023-04-20 23:58:07'))

In [30]:
(test_intervals['finish'] - test_intervals['start']).describe()

count                          189
mean     0 days 04:59:35.428571428
std      0 days 03:06:42.587346539
min                0 days 02:42:34
25%                0 days 03:38:37
50%                0 days 04:13:31
75%                0 days 05:05:08
max                1 days 00:58:29
dtype: object

Тестовый период - с 01.01.2022 по 20.04.2023 включительно. В среднем, интервал предсказания составляет 5 часов, медиана - 4 часа 13 минут. Максимальный горизонт предсказания - 26 часов.

#### 2.3.2. X_test

В наборе сигналов тестовой выборки (X_test) присутствуют интервалы с пропущенными значениями. Список интервалов предоставлен в test_intervals. Для этих интервалов необходимо предсказать наличие неисправности.

In [31]:
X_test.head()

,4 ВИБРАЦИЯ НА ОПОРЕ 1,4 ВИБРАЦИЯ НА ОПОРЕ 2,4 ВИБРАЦИЯ НА ОПОРЕ 3,4 ВИБРАЦИЯ НА ОПОРЕ 3 ПРОДОЛЬНАЯ,4 ВИБРАЦИЯ НА ОПОРЕ 4,4 ВИБРАЦИЯ НА ОПОРЕ 4 ПРОДОЛЬНАЯ,4 ДАВЛЕНИЕ МАСЛА В СИСТЕМЕ,4 ТЕМПЕРАТУРА МАСЛА В МАСЛОБЛОКЕ,4 ТЕМПЕРАТУРА МАСЛА В СИСТЕМЕ,4 ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 1,4 ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 2,4 ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 3,4 ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 4,4 ТОК РОТОРА 1,4 ТОК РОТОРА 2,4 ТОК СТАТОРА
dt,,,,,,,,,,,,,,,,
2022-01-01 00:00:00,0.758333,1.180000,0.360000,1.906000,2.075000,1.416667,134.444000,NaN,31.235,42.595,43.540,30.830,34.394,247.51,247.51,271.153333
2022-01-01 00:01:00,0.738000,1.211667,0.374000,1.913333,1.780000,1.291667,134.420000,33.260,31.100,42.595,43.405,31.235,34.610,247.39,247.39,271.195000
2022-01-01 00:02:00,0.730000,1.176000,0.346000,1.880000,2.096667,1.435000,134.375000,32.720,31.235,42.676,43.000,NaN,34.340,247.51,247.51,271.441667
2022-01-01 00:03:00,0.702000,1.212000,0.323333,1.974000,1.596667,1.358000,134.446000,NaN,31.640,42.595,43.405,31.235,34.610,247.51,247.51,270.730000
2022-01-01 00:04:00,0.676667,1.168333,0.318000,1.855000,1.503333,1.316667,134.453333,32.855,NaN,42.514,NaN,31.235,34.475,247.51,247.51,271.562000
